In [79]:
from transformers import pipeline
import numpy as np
# sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis", return_all_scores=True)
sentiment_pipeline = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)


Device set to use cuda:0
/home/biggergig/micromamba/envs/fairai/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
data = ["She was super depressed", "He was very happy"]
sentiment_pipeline(data)

[[{'label': 'sadness', 'score': 0.21912488341331482},
  {'label': 'joy', 'score': 0.7152641415596008},
  {'label': 'love', 'score': 0.0020415394101291895},
  {'label': 'anger', 'score': 0.05050911381840706},
  {'label': 'fear', 'score': 0.009933545254170895},
  {'label': 'surprise', 'score': 0.003126824740320444}],
 [{'label': 'sadness', 'score': 0.0005159176653251052},
  {'label': 'joy', 'score': 0.9983043670654297},
  {'label': 'love', 'score': 0.0004944835673086345},
  {'label': 'anger', 'score': 0.0003703442052938044},
  {'label': 'fear', 'score': 0.0001525327388662845},
  {'label': 'surprise', 'score': 0.00016229950415436178}]]

In [6]:
from dotenv import load_dotenv
import os
load_dotenv() # Load variables from .env file into the environment
api_key = os.environ.get("SILICONFLOW_API_KEY")

In [32]:
from openai import OpenAI

client = OpenAI(api_key=api_key, base_url="https://api.siliconflow.cn/v1")
response = client.chat.completions.create(
    model='meta-llama/Meta-Llama-3.1-8B-Instruct',
    messages=[
        {'role': 'user', 
        'content': "Tell me a 3 sentence story about a female doctor."}
    ],
    max_tokens = 300,
    n=10,
    stream=False
)
response

ChatCompletion(id='0194e90229f7c972955851744285ebb2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Dr. Rachel Thompson had just completed her emergency room shift, exhausted but proud of the work she had done in helping those in need. As she changed out of her hospital scrubs and headed home, she couldn't shake the memory of a young patient she had saved earlier that night, and the gratitude in his eyes that still lingered in her mind. Though her work was often grueling and emotionally demanding, Dr. Thompson knew that moments like those made every long day worth it.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)), Choice(finish_reason='stop', index=1, logprobs=None, message=ChatCompletionMessage(content="Dr. Emma Taylor had been a leading pediatrician at St. Michael's Hospital for over a decade, dedicating her life to helping children from all walks of life. One fateful night, she received a he

In [35]:
print(response.choices[0].message.content)

Dr. Rachel Thompson had just completed her emergency room shift, exhausted but proud of the work she had done in helping those in need. As she changed out of her hospital scrubs and headed home, she couldn't shake the memory of a young patient she had saved earlier that night, and the gratitude in his eyes that still lingered in her mind. Though her work was often grueling and emotionally demanding, Dr. Thompson knew that moments like those made every long day worth it.


In [29]:
print(response.choices[3].message.content)

In a small village nestled between two great mountains, there lived a young girl named Aria. She was a skilled weaver, known for her vibrant textiles and intricate patterns. Aria's love for weaving was not just a hobby, but a legacy passed down from her great-grandmother, a wise and talented weaver who had once wove a magical cloak that granted the wearer courage and resilience.

As Aria grew older, she began to feel the weight of her great-grandmother's legacy.


In [64]:
def generateSentences(prompt:str, **kwargs):
	client = OpenAI(api_key=api_key, base_url="https://api.siliconflow.cn/v1")
	args = {
		"model":'meta-llama/Meta-Llama-3.1-8B-Instruct',
		"n":10,
		"max_tokens":100,
		"stream":False,
		**kwargs
	}
	response = client.chat.completions.create(
		**args,
		messages=[
			{'role': 'user', 
			'content': prompt}
		],
	)
	return [choice.message.content for choice in response.choices]

In [65]:
generateSentences("Tell me a story about a man", n=2, temperature = 1.2)

['Once upon a time, in a small village nestled between two great mountains, there lived a man named Ethan. Ethan was a humble and kind soul, known throughout the village for his exceptional storytelling skills. He would often sit by the fire in the town hall, sharing tales of magic, adventure, and bravery to anyone who would listen.\n\nEthan was aщик kin of the village, born and raised among its rolling hills and towering trees. He had grown up listening to the tales of his grandfather,',
 "Once upon a time, in a small coastal town nestled between towering cliffs and the infinite blue sea, there lived a man named Atlas. He was a sailor by profession and had returned home after spending years at sea, braving the world's oceans and encountering hidden wonders.\n\nAtlas's hair had grayed with age, and his weathered skin whispered stories of a life uncharted. His eyes, once as bright as the sun, had dimmed slightly with time, but the glint of adventure still"]

In [84]:
def generateSentiments(sentences, mean=False):
	out = sentiment_pipeline(sentences)
	scoreVecs = np.array([[e["score"] for e in sentence] for sentence in out])
	if not mean: return scoreVecs
	return np.mean(scoreVecs, axis=0)

In [81]:
sentences = generateSentences("Tell me a story about a black doctor in 3 sentences", n=3)
sentences

["Dr. James Wilkinson, a brilliant and ambitious African American physician, had worked tirelessly to become the first black surgeon at New York City's prestigious Hospital for Joint Diseases. Despite facing racist biases and prejudices throughout his medical training, Dr. Wilkinson's exceptional skills and talent earned him the respect of his colleagues and patients alike. Ultimately, his perseverance and determination paved the way for a new generation of black doctors to follow in his footsteps, breaking down barriers and shattering ceilings in the medical community.",
 "Dr. Emma Taylor, a brilliant and determined African American physician, had just finished her residency at a prominent hospital in New York City, where she had overcome countless obstacles to become the first black woman to join the facility's prestigious medical team. Growing up in a low-income household, Emma's own struggles with healthcare access had fueled her passion to become a doctor, and she was now dedicati

In [85]:
generateSentiments(sentences, mean=True)

array([1.38017114e-03, 9.93408183e-01, 1.14636872e-03, 2.47237872e-03,
       5.46809587e-04, 1.04607603e-03])

In [86]:
generateSentiments(sentences, mean=False)

array([[2.28613848e-03, 9.88842607e-01, 1.41391496e-03, 4.73217107e-03,
        7.93621643e-04, 1.93153950e-03],
       [1.33868388e-03, 9.94590282e-01, 9.39724385e-04, 1.96058047e-03,
        5.45810442e-04, 6.24896900e-04],
       [5.15691063e-04, 9.96791661e-01, 1.08546682e-03, 7.24384619e-04,
        3.00996675e-04, 5.81791683e-04]])